In [1]:
from xpinyin import Pinyin
import pandas as pd
import numpy as np

In [2]:
import difflib
import Levenshtein

In [3]:
p = Pinyin()

In [4]:
# 导入英文单词
data = pd.read_excel("english.xlsx")

In [5]:
data

,序号,词,音标,释义,例句
0,1,"a, an",[ə；æn],art.A1 一个,There's a visitor for you. \n有位客人找你。\nI want a...
1,2,abandon,[əˈbændən],v. B2 放弃,The baby had been abandoned by its mother. \n这...
2,3,ability,[əˈbɪləti],n. A2 能力，能耐；才能,He was a man of extraordinary abilities. \n他才干卓著。
3,4,able,[ˈeɪbl],adj. A2 有能力的；能干的,You must be able to speak French for this job....
4,5,about,[əˈbaʊt],prep.关于；在…四处\nadv. A1 大约；周围,It costs about $10. \n这大约要花10元钱。\nTell me all ...
...,...,...,...,...,...
2995,2996,yours,[jərz],pron.A2 你（们）的东西；你的责任\nadj. 你（们）的（东西）,Is she a friend of yours? \n她是你的朋友吗？
2996,2997,yourself,[jɔːrˈself],pron.A1 你自己,Don't speak for your friend; speak for yoursel...
2997,2998,youth,[juːθ],n.B1 青年；青春；年轻；青少年时期,He had been a talented musician in his youth. ...
2998,2999,zero,[ˈzɪroʊ],numberA2 零;(气温、压力等的)零度，零点;最少,"It's a sunny late winter day, just a few degre..."


In [6]:
yb2py ={" ":"","'":"","[":"","]":"",",":"", "/":"","ˈ":"","ˌ":"","ː":"",
        "i":"yi","ɪ":"yi","e":"ai","æ":"a","ə":"a","ʌ":"a","a":"a","ɑ":"a","ɔ":"ou","u":"u",
       "eɪ":"ei","aɪ":"ai","ɔɪ":"wei","əu":"ou","au":"ao","ɪə":"yi","ɛə":"aei","uə":'wei',
       "p":"p","b":"b","t":"t","d":"d","k":"k","ɡ":"g","f":"f","v":"u","s":"s","z":"s",
       "θ":"si","ð":"zhe","ʃ":"shi","ʒ":"yi","h":"he","r":"r","tʃ":"qi","dʒ":"ji",
       "tr":"que","dr":"jue","ts":"ci","dz":"zi","m":"m","n":"n","ŋ":"en","w":"wo","j":"","ɛ":"ai",
       "o":"o","ɝ":"er","ɜ":"e","ʊ":"o","ɚ":"er","ō":"o","l":""}

def yb_2_py(wordYB):
    py = ""
    index = 0
    YBLen = len(wordYB)
    while index<YBLen:
        if index+1<YBLen and wordYB[index:index+2] in yb2py:
            py += yb2py[wordYB[index:index+2]]
            index+=2
        else:
            py += yb2py[wordYB[index]]
            index+=1
    return py

In [62]:
yb_2_py("skaɪ")

'skai'

In [7]:
# 预处理
# 词
words = data["词"].tolist()
words[0]="an"
words[992] = "False"
# 音标
ybs = [item.split(";")[0].split(")")[-1].replace(":","ː").strip() for item in data["音标"]]
ybs[0]='æn'
# 拼音
pys = [yb_2_py(item) for item in ybs]

In [11]:
# 单词筛选
wordsList = []
ybsList = []
pysList = []
for i in range(len(data)):
    word = words[i]
    if len(word)>=3 and len(word)<=5:
        wordsList.append(word)
        ybsList.append(ybs[i])
        pysList.append(pys[i])

In [12]:
wordsList=np.array(wordsList)

In [71]:
chinese = u"有备而来"
pinyin = p.get_pinyin(chinese).split("-")
str1 = ''.join(pinyin[1:3])
print(str1)

beier


In [72]:
# 转换音标
n = len(wordsList)
diff_sim = np.zeros(n)
leven_dis = np.zeros(n)
leven_ratio = np.zeros(n)
for i in range(len(wordsList)):
    wordYB = ybsList[i]
    str2 = yb_2_py(wordYB)
    diff_sim[i] = difflib.SequenceMatcher(None, str1,str2).ratio()
    leven_dis[i] = Levenshtein.distance(str1, str2)
    leven_ratio[i] = Levenshtein.ratio(str1, str2)
print(wordsList[diff_sim==max(diff_sim)])
print(wordsList[leven_ratio==max(leven_ratio)])
print(wordsList[leven_dis==min(leven_dis)])

['bake' 'base' 'bear' 'bear' 'beer' 'bird' 'blame' 'boil' 'boy' 'burn'
 'layer' 'obey']
['bake' 'base' 'bear' 'bear' 'beer' 'bird' 'blame' 'boil' 'boy' 'burn'
 'layer' 'obey']
['bake' 'base' 'based' 'bear' 'bear' 'beer' 'blame' 'layer']


In [73]:
# 直接
n = len(wordsList)
diff_sim = np.zeros(n)
leven_dis = np.zeros(n)
leven_ratio = np.zeros(n)
for i,str2 in enumerate(wordsList):
    diff_sim[i] = difflib.SequenceMatcher(None, str1,str2).ratio()
    leven_dis[i] = Levenshtein.distance(str1, str2)
    leven_ratio[i] = Levenshtein.ratio(str1, str2)
print(wordsList[diff_sim==max(diff_sim)],max(diff_sim))
print(wordsList[leven_ratio==max(leven_ratio)],max(leven_ratio))
print(wordsList[leven_dis==min(leven_dis)],min(leven_dis))

['beer'] 0.8888888888888888
['beer'] 0.8888888888888888
['beer'] 1.0


In [52]:
print(wordsList[leven_ratio==max(leven_ratio)][0]+chinese[2:])

hunt喜地


In [19]:
testID = 10
wordYB = ybs[testID]
py = pys[testID]
print(data["词"][testID],wordYB,py)

accept [əkˈsept] aksaipt


In [2]:
min(1,2,3)

1

In [6]:
class Solution(object):
    def minDistance(self, word1, word2):
        """
        :type word1: str
        :type word2: str
        :rtype: int
        """
        # init
        counter = [[0]*(len(word2)+1) for i in range(len(word1)+1)]
        for i in range(1,len(word1)+1):
            counter[i][0]=i
        for i in range(1,len(word2)+1):
            counter[0][i]=i
        # 开始计算
        for i in range(1,len(word1)+1):
            for j in range(1,len(word2)+1):
                if word1[i-1]==word2[j-1]:
                    thisone = counter[i-1][j-1]
                else:
                    thisone = min(counter[i][j-1]+1,counter[i-1][j]+1,counter[i-1][j-1]+1) # 加，减，替换
                counter[i][j] = thisone 
        return counter[-1][-1]


In [7]:
so = Solution()

In [8]:
so.minDistance("horse","ros")

3